# Generador de archivos para llenar buckets en GCP

Imports

In [ ]:
import pandas as pd
import random as rd
import string as st
import os
import google.cloud.storage as storage
from datetime import datetime
from datetime import timedelta

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Constantes

In [ ]:
# Rutas
path = '/content/drive/Shareddrives/Infra_Grupo_Sarasa/repositorio/data/TP_Final_Especificacion_1.xlsx'
path_coord ='/content/drive/Shareddrives/Infra_Grupo_Sarasa/repositorio/data/ar.csv'
path_destino = '../data'

# Cantidad de días desde la fecha actual hacia el pasado que generamos datos
cant_dias = 2

# Fecha actual
fecha_actual = datetime.now()

# Cantidad de distribuidores
cant_dist = 5 # 4 - 1

# Cantidad de clientes
cant_clientes = 3

# Cantidad de sucursales
cant_sucursales = 5

Usamos credenciales e instanciamos storage cliente

In [ ]:
credentials = '/content/drive/Shareddrives/Infra_Grupo_Sarasa/repositorio/usm-infra-grupo1-42c8c1c4178a.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials
storage_client = storage.Client()

Leemos hojas de planilla excel, generamos dataframes de cada una y leemos csv de coordenadas

In [ ]:
df_sheets_desc = pd.read_excel(path, sheet_name=None)
df_cond_vta = df_sheets_desc['Condicion_Venta']
df_unidades = df_sheets_desc['Unidades']
df_tipo_negocio = df_sheets_desc['Tipo_Negocio']
df_cadena = df_sheets_desc['Cadena']
df_dias_visita = df_sheets_desc['Dias_Visita']
df_productos = df_sheets_desc['Productos']
df_cond_venta = df_sheets_desc['Condicion_Venta']
df_localidades = df_sheets_desc['Prov_Localidades']
df_estado = df_sheets_desc['Estado_cliente']
df_dias_visita = df_sheets_desc['Dias_Visita']

coordenadas = pd.read_csv(path_coord)

Funciones útiles

In [ ]:
def generar_codigos_sku(productos):
  codigos_sku = {}
  for producto in productos:
    caracteres = st.ascii_uppercase + st.digits
    sku_codigo = ''.join(rd.choice(caracteres) for _ in range(10))
    codigos_sku[producto] = sku_codigo

  return codigos_sku

In [ ]:
def generate_fecha_cierre(fecha_cierre: datetime) -> str:
    """
    Genera una fecha de cierre de comprobante aleatoria entre la fecha de cierre
    de comprobante y la fecha actual.
    """
    fech_cie_com = (fecha_cierre - timedelta(hours=rd.randint(0, 23), minutes=rd.randint(0, 59), seconds=rd.randint(0, 59))) \
            .strftime('%Y-%m-%d %H:%M:%S')
    return fech_cie_com

In [ ]:
def generate_data_per_days(
        days: int,
        distribuidor: int,
        df_cond_vta: pd.DataFrame,
        df_localidades: pd.DataFrame,
        df_dias_visita: pd.DataFrame,
        df_productos: pd.DataFrame,
        df_sheets_desc: dict,
        df_tipo_negocio: pd.DataFrame,
        df_estado: pd.DataFrame,
        coordenadas: pd.DataFrame,
    ) -> tuple[dict, str]:
    fech_cierre = fecha_actual - timedelta(days=days)
    fech_cierre_string = fech_cierre.strftime('%Y-%m-%d')
    fech_cie_com_list = []
    vta_unidades_list = []
    vta_importe_list = []
    cond_vta_list = []
    sucursales_dist_list = []
    clientes_list = []
    provincias_list = []
    ciudades_list = []
    estado_list = []
    nombre_clientes_list = []
    cuit_list = []
    razon_social_list = []
    direccion_list = []
    dias_visita_list = []
    telefono_list = []
    fecha_alta_list = []
    fecha_baja_list = []
    tipo_negocio_list = []
    unidad_list = []
    stock_list = []
    deuda_vencida_list = []
    deuda_total_list = []


    # Generar SKUs a partir de la lista de productos
    codigos_sku = generar_codigos_sku(df_productos['nombre'])

    # Nombre de las columnas de los archivos cliente,venta,stock
    columns_v = df_sheets_desc['Desc_Ventas']['Campo']
    columns_s = df_sheets_desc['Desc_Stock']['Campo']
    columns_m = df_sheets_desc['Desc_Clientes']['Campo']
    columns_d = df_sheets_desc['Desc_Deuda']['Campo']

    for cliente_nro in range(cant_clientes):
        codigo_sucursal_base = (distribuidor * 100)  # Código base para el distribuidor
        fech_cie_com_list.append(generate_fecha_cierre(fecha_cierre=fech_cierre))
        vta_unidades_list.append(rd.randint(0,100))
        vta_importe_list.append(round((rd.uniform(100, 1000)), 2))
        cond_vta_list.append(rd.choice(df_cond_vta['codigo_condicion']))
        sucursales_dist_list.append(codigo_sucursal_base + rd.randint(10, 99))
        clientes_list.append(int(f'{codigo_sucursal_base}'+ f'{rd.randint(1000, 9999)}')) #modificación para clientes únicos taggeados por sucursal
        provincias_list.append(df_localidades.columns.values[distribuidor%len(df_localidades.columns.values)])
        ciudades_list = [rd.choice(df_localidades[p].dropna().tolist()) for p in provincias_list]
        estado = rd.choices(df_estado['nombre'], weights=df_estado['probabilidades'], k=1)[0]
        estado_list.append(estado)
        nombre_clientes_list.append(f'Cliente_{cliente_nro}')
        cuit_list.append(rd.randint(10000000, 99999999))
        razon_social_list.append(f'Empresa_{cliente_nro}')
        direccion_list.append(f'Direccion_{cliente_nro}')
        dias_visita_list.append(rd.choice(df_dias_visita['codigo_dia']))
        telefono_list.append(f"{rd.randint(100, 999)}-{rd.randint(100, 999)}-{rd.randint(1000, 9999)}")
        if estado == "I":
            fecha_baja_list.append((fech_cierre + timedelta(days=rd.randint(cant_dias,365))).strftime('%Y-%m-%d'))
        else:
            fecha_baja_list.append(pd.NA)
        fecha_alta_list.append((fech_cierre - timedelta(days=rd.randint(0,cant_dias))).strftime('%Y-%m-%d'))
        tipo_negocio_list.append(rd.choice(df_tipo_negocio['nombre']))
        unidad = rd.choice(df_unidades['codigo_unidad'])
        unidad_list.append(unidad)
        stock_list.append(rd.randint(100, 500) if unidad == "UNI" else rd.randint(1, 100))
        deuda_vencida_list.append(round(rd.uniform(0, 1000), 2))
        deuda_total_list.append(round(rd.uniform(0, 10000), 2))
    distribuidor_list = [distribuidor for i in range(cant_clientes)]


    coordenada_latitud = []
    coordenada_longitud = []
    for ciudad in ciudades_list:
        coordenada_latitud.append(list(coordenadas[coordenadas['city'] == ciudad]['lat'])[0])
        coordenada_longitud.append(list(coordenadas[coordenadas['city'] == ciudad]['lng'])[0])

    #Producto
    producto=[]
    producto1=[]
    producto1 = list(codigos_sku.keys())
    producto1.extend(producto1)
    producto=producto1

    #SKU
    sku_codigo=[]
    sku_codigo1=[]
    sku_codigo1 = list(codigos_sku.values())
    sku_codigo1.extend(sku_codigo1)
    sku_codigo = sku_codigo1

    #data frame con data
    venta_clientes_data = list(
        zip(
            sucursales_dist_list,
            clientes_list,
            fech_cie_com_list,
            sku_codigo,
            vta_unidades_list,
            vta_importe_list,
            cond_vta_list,
            distribuidor_list
        )
    )
    stock_data = list(
        zip(
            sucursales_dist_list,
            fech_cie_com_list,
            sku_codigo,
            producto,
            stock_list,
            unidad_list,
            distribuidor_list
        )
    )
    clientes_data = list(
        zip(
            sucursales_dist_list,
            clientes_list,
            ciudades_list,
            provincias_list,
            estado_list,
            nombre_clientes_list,
            cuit_list,
            razon_social_list,
            direccion_list,
            dias_visita_list,
            telefono_list,
            fecha_alta_list,
            fecha_baja_list,
            coordenada_latitud,
            coordenada_longitud,
            cond_vta_list,
            deuda_vencida_list,
            tipo_negocio_list,
            distribuidor_list
        )
    )
    deuda_data = list(
        zip(
            sucursales_dist_list,
            clientes_list,
            fech_cie_com_list,
            deuda_vencida_list,
            deuda_total_list,
            distribuidor_list
        )
    )

    df_vta_cli = pd.DataFrame(venta_clientes_data,columns=columns_v)
    df_stock = pd.DataFrame(stock_data,columns=columns_s)
    df_clientes = pd.DataFrame(clientes_data,columns=columns_m)
    df_deuda =  pd.DataFrame(deuda_data,columns=columns_d)

    print(fecha_cierre)
    print('DEUDA',deuda_vencida_list)
    print('CLIENTE',clientes_list)

    return deuda_vencida_list,clientes_list

    return { 'venta': df_vta_cli, 'stock': df_stock, 'cliente': df_clientes, 'deuda': df_deuda }, fech_cierre_string


In [ ]:

for distribuidor in range(1, cant_dist):
    carpeta_destino = 'g1_distribuidor_' + str(distribuidor)
    bucket = storage_client.bucket(carpeta_destino)

    if not bucket.exists():
        bucket = storage_client.create_bucket(carpeta_destino)

    for dias in range(0, cant_dias):
        dfDict, fecha_cierre = generate_data_per_days(
            days=dias,
            distribuidor=distribuidor,
            df_cond_vta=df_cond_vta,
            df_localidades=df_localidades,
            df_dias_visita=df_dias_visita,
            df_productos=df_productos,
            df_sheets_desc=df_sheets_desc,
            df_tipo_negocio=df_tipo_negocio,
            df_estado=df_estado,
            coordenadas=coordenadas,
        )

        for key, df in dfDict.items():
            archivo_destino = f'{key}/{fecha_cierre}.csv'
            blob = bucket.blob(archivo_destino)
            blob.upload_from_string(df.to_csv(index=False), 'text/csv')

2023-11-01
DEUDA [250.54, 297.98, 705.54]
CLIENTE [1007083, 1003300, 1001605]
[1007083, 1003300, 1001605]
DEUDA [454.41, 280.16, 875.03]
CLIENTE [1003660, 1009318, 1002758]
[1003660, 1009318, 1002758]
DEUDA [618.78, 962.71, 418.2]
CLIENTE [2002106, 2007046, 2002315]
[2002106, 2007046, 2002315]
DEUDA [138.77, 487.95, 488.15]
CLIENTE [2003609, 2008885, 2002578]
[2003609, 2008885, 2002578]
DEUDA [361.53, 239.94, 226.42]
CLIENTE [3006455, 3007093, 3002591]
[3006455, 3007093, 3002591]
DEUDA [772.03, 53.03, 608.44]
CLIENTE [3006692, 3001090, 3005761]
[3006692, 3001090, 3005761]
DEUDA [482.58, 536.35, 354.37]
CLIENTE [4003693, 4009642, 4008830]
[4003693, 4009642, 4008830]
DEUDA [628.21, 315.87, 468.92]
CLIENTE [4003680, 4001312, 4002068]
